In [ ]:
# Импортируем все необходимые библиотеки

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import re
import unicodedata
from typing import Tuple
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

In [ ]:
# Загружаем переменные окружения из .env файла
load_dotenv()

In [ ]:
# Создаем URL для SQLAlchemy
SQLALCHEMY_DATABASE_URL = (
    f"postgresql://"
    f"{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@"
    f"{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/"
    f"{os.getenv('POSTGRES_DATABASE')}"
)
engine = create_engine(SQLALCHEMY_DATABASE_URL)

### **Выгрузим три таблицы из базы данных**

1. Таблица `user_data`

```python
# Структура таблицы:
user_data = {
    "user_id": "Уникальный идентификатор пользователя (int64)",
    "gender": "Пол пользователя (int: 0/1)",
    "age": "Возраст пользователя (int32, 0-99)",
    "country": "Страна пользователя (str: 'Russian', 'Belarus', 'Iran', ...)",
    "city": "Город пользователя (str: 'Moscow', 'Abakan', ...)",
    "exp_group": "Экспериментальная группа (int8: 0-4)",
    "os": "Операционная система (str: 'iOS', 'Android', 'web')",
    "source": "Источник привлечения (str: 'ads', ...)"
}
```

2. Таблица `post_text_df`

```python
# Структура таблицы:
post_data = {
    "post_id": "Уникальный ID поста (int64)",
    "text": "Текст поста (str)",
    "topic": "Категория (str: 'politics', 'entertainment', ...)",
}
```

3. Таблица `feed_data`

```python
# Структура таблицы:
feed_data = {
    "user_id": "ID юзера (int64)",
    "post_id": "ID поста (int64)",
    "text": "Текст поста (str)",
    "action": "view/like (str)",
    "timestamp": "Время просмотра поста (timestamp)",
    "target": "0/1 (int64): 1 - если пост был лайкнут в ближайшее время после просмотра (условно в первые 1-2 минуты)",

}
```


In [ ]:
# Данные по пользователям

user_info = pd.read_sql("""SELECT * FROM user_data LIMIT 10""", con=engine)

user_info.head()

In [ ]:
print(user_info.columns)

In [ ]:
# Посты и топики

posts_info = pd.read_sql("""SELECT * FROM post_text_df""", con=engine)

posts_info.head()

In [ ]:
# Попробуем забрать, скажем, 5 миллионов

feed_data = pd.read_sql(
    """SELECT * FROM feed_data WHERE action = 'view' LIMIT 500""", con=engine
)

feed_data.head()

In [ ]:
# Проверяем распределение классов

print("Распределение классов:")
print(feed_data["target"].value_counts(normalize=True))

## Feature Engineering для таблицы `post_text_df`

### 📌 План генерации признаков

#### 1. Базовые текстовые метрики

**Генерируемые фичи:**

- `text_length` - длина текста в символах
- `word_count` - количество слов в тексте
- `digit_count` - количество цифр в тексте

**Цель:**  
Выявить зависимость между структурой текста и пользовательской вовлеченностью.

---

#### 2. TF-IDF векторизация с предобработкой

**Этапы:**

1. Предобработка текста с помощью spaCy:
   - Лемматизация
   - Удаление стоп-слов
   - Удаление пунктуации/чисел
2. Построение TF-IDF матрицы (10000 наиболее значимых терминов)

**Особенности:**

- Используется русскоязычная модель `ru_core_news_sm`
- Минимальная длина токена - 5 символа

---

#### 3. Агрегатные статистики TF-IDF

**Генерируемые фичи:**

- `tfidf_mean` - среднее значение TF-IDF по терминам
- `tfidf_sum` - сумма значений TF-IDF
- `tfidf_max` - максимальное значение TF-IDF

**Гипотеза:**  
Агрегатные характеристики помогут:

- Оценить общую информативность поста (`mean`)
- Измерить суммарный "вес" ключевых слов (`sum`)
- Выявить доминирующий термин (`max`)

#### PS: В конце поймем, что особо ничего полезного эти фичи не принесли :(

---

#### 4. Снижение размерности

**Метод:** TruncatedSVD (300 компонент)

**Преимущества:**

- Стараемся сохранить как можно больше дисперсии данных
- Уменьшает шумовые компоненты
- Оптимизирует вычислительные затраты для последующей кластеризации

---

#### 5. Кластеризация текстов

**Параметры:**

- Метод: K-Means
- Количество кластеров: 25
- Дополнительные фичи: расстояния до центроидов всех кластеров

**Зачем нужны расстояния:**

- Показывают степень принадлежности к кластерам
- Характеризуют положение поста в признаковом пространстве
- Позволяют выявлять аномальные тексты

---

#### 6. Метрики популярности

**Источник данных:** таблица `feed_data`

**Рассчитываемые показатели:**

- `views` - общее количество просмотров
- `likes` - количество лайков
- `ctr` - коэффициент конверсии (лайки/просмотры)

**Аналитическая ценность:**

- Возможность связать текстовые характеристики с пользовательским engagement
- Выявление "идеального" профиля поста

---


In [ ]:
# Загрузка модели spaCy
nlp = spacy.load("en_core_web_sm")


### Функция для предобработки текста
def spacy_preprocess(text: str) -> Tuple[str, int]:
    """
    Очищает текст и унифицирует шрифт:
    - Убирает пунктуацию
    - Группирует числа в один признак (количество чисел)
    - Приводит текст к нижнему регистру
    - Лемматизирует слова
    - Удаляет стоп-слова и короткие слова
    - Унифицирует шрифт (например, убирает стильные шрифты)
    """
    # 1. Унификация шрифта
    text = unicodedata.normalize("NFKC", text)

    # 2. Удаляем HTML-теги и эмодзи
    text = re.sub(r"<[^>]+>", " ", text)  # HTML-теги
    text = re.sub(r"[\U00010000-\U0010ffff]", " ", text)  # Эмодзи

    # 3. Убираем пунктуацию и лишние пробелы
    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    # 4. Заменяем числа на <NUMBER>
    numbers_count = len(re.findall(r"\d+", text))
    text = re.sub(r"\d+", "<NUMBER>", text)

    # 5. Приводим к нижнему регистру
    text = text.lower()

    # 6. Лемматизация и фильтрация
    doc = nlp(text)
    tokens = [
        token.lemma_
        for token in doc
        if token.is_alpha and not token.is_stop and len(token) > 2
    ]

    return " ".join(tokens), numbers_count

In [ ]:
# Применяем spaCy очистку
posts_info["numbers_count"] = posts_info["text"].apply(lambda x: spacy_preprocess(x)[1])
posts_info["cleaned_text"] = posts_info["text"].apply(lambda x: spacy_preprocess(x)[0])

In [ ]:
# Функция для подсчета количества слов
def count_words(text):
    if isinstance(text, str):  # Проверяем, что текст не NaN
        words = re.findall(
            r"\b\w+\b", text
        )  # Ищем только слова (без знаков препинания)
        return len(words)
    return 0


# Добавляем новые столбцы
posts_info["text_length"] = posts_info["text"].apply(len)  # Длина текста
posts_info["word_count"] = posts_info["text"].apply(count_words)  # Количество слов

In [ ]:
# TF-IDF векторизация
vectorizer_spacy = TfidfVectorizer(
    max_features=None,  # не ограничивать
    stop_words="english",
    min_df=5,
    max_df=0.95,  # удалим очень частые (шум)
    use_idf=True,
)
tfidf_spacy_matrix = vectorizer_spacy.fit_transform(
    posts_info["cleaned_text"]
).toarray()

# Преобразуем в DataFrame
tfidf_spacy_df = pd.DataFrame(
    tfidf_spacy_matrix, columns=vectorizer_spacy.get_feature_names_out()
)
tfidf_spacy_df["post_id"] = posts_info["post_id"].values  # добавим идентификатор

tfidf_spacy_df.head()

In [ ]:
### Генерим фичи на основе TfIdf (возьмем сначала простую агрегацию)
posts_info["TotalTfIdf"] = (
    tfidf_spacy_df.drop(columns=["post_id", "topic"], axis=1)
    .sum(axis=1)
    .reset_index()[0]
)
posts_info["MaxTfIdf"] = (
    tfidf_spacy_df.drop(columns=["post_id", "topic"], axis=1)
    .max(axis=1)
    .reset_index()[0]
)
posts_info["MeanTfIdf"] = (
    tfidf_spacy_df.drop(columns=["post_id", "topic"], axis=1)
    .mean(axis=1)
    .reset_index()[0]
)

posts_info.head()

In [ ]:
### Снизим размерность TF-IDF матрицы
centered = tfidf_spacy_matrix - tfidf_spacy_matrix.mean()

# TruncatedSVD
svd = TruncatedSVD(n_components=300, random_state=42)
svd_decomp = svd.fit_transform(centered)

In [ ]:
### Кластеризация по семантике
kmeans = KMeans(n_clusters=25, random_state=42)
cluster = kmeans.fit_predict(svd_decomp).reshape(-1, 1)
posts_info["TextCluster"] = cluster

# Генерация названий столбцов динамически
num_clusters = 25  # Количество кластеров
dists_columns = [
    f"DistanceTo{cluster}thCluster" for cluster in range(1, num_clusters + 1)
]

# Создание DataFrame с автоматически сгенерированными названиями столбцов
dists_df = pd.DataFrame(data=kmeans.transform(svd_decomp), columns=dists_columns)

# Проверка первых строк
dists_df.head()

In [ ]:
posts_info = pd.concat((posts_info, dists_df), axis=1).drop(
    columns=["cleaned_text"], axis=1
)

In [ ]:
post_statistics = pd.read_sql(
    """
    WITH prep AS (
        SELECT
            timestamp,
            user_id,
            post_id,
            action,
            target,
            MIN(timestamp) FILTER (WHERE target = 1) OVER (PARTITION BY user_id, post_id) AS min_timestamp,
            CASE
                WHEN timestamp > MIN(timestamp) FILTER (WHERE target = 1) OVER (PARTITION BY user_id, post_id)
                THEN 1
                ELSE 0
            END AS check
        FROM feed_data
        WHERE action <> 'like'
    )
    SELECT
        post_id,
        COUNT(*) FILTER (WHERE target = 0) AS total_views_post,
        COUNT(*) FILTER (WHERE target = 1) AS total_likes_post,
        COUNT(*) FILTER (WHERE target = 1) * 1.0 / NULLIF(COUNT(*), 0) AS post_ctr
    FROM prep
    WHERE "check" = 0
    GROUP BY post_id
    """,
    con=engine,
)

# Заполняем медианами
views_median = post_statistics["total_views_post"].median()
likes_median = post_statistics["total_likes_post"].median()

post_statistics["total_views_post"].fillna(views_median, inplace=True)
post_statistics["total_likes_post"].fillna(likes_median, inplace=True)

# Пересчитываем CTR
post_statistics["post_ctr"] = (
    post_statistics["total_likes_post"] / post_statistics["total_views_post"]
)

post_statistics.head()

In [ ]:
posts_info = posts_info.join(
    post_statistics.set_index("post_id"), on="post_id", how="left"
)

In [ ]:
posts_info.to_sql(
    "posts_info_features_ruslan_prashchurovich",
    con=engine,
    index=False,
    if_exists="replace",
)

In [ ]:
# Функция для пакетной загрузки данных из SQL
def batch_load_sql(query: str, engine) -> pd.DataFrame:
    CHUNKSIZE = 200000
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)


# Функция для загрузки признаков из базы данных
def load_features(engine) -> pd.DataFrame:
    query = "SELECT * FROM posts_info_features_ruslan_prashchurovich"
    return batch_load_sql(query, engine)


# Проверка загрузки данных
post_features = load_features(engine)
post_features.head()

In [ ]:
### Почистим переменные

del svd
del KMeans
del spacy_preprocess

del tfidf_spacy_matrix
del tfidf_spacy_df
del centered
del svd_decomp
del kmeans
del dists_columns
del dists_df

import gc

gc.collect()